### Installing and Importing necessary packages

In [1]:
import pandas as pd
from pycorenlp import StanfordCoreNLP
from azure.cognitiveservices.speech import AudioDataStream, SpeechConfig, SpeechSynthesizer, SpeechSynthesisOutputFormat
from azure.cognitiveservices.speech.audio import AudioOutputConfig
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk.corpus
from nltk.corpus import nps_chat
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('nps_chat')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


True

### Making necessary connections with Google API and Anvil server

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bionic-genre-326715-416d6d51b533.json"
import anvil.server
anvil.server.connect("XBQM36YNDUZ4TWI522HM4KDL-NUKYRRNENRSFFQ4A")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


### Speech to text conversion

In [3]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file, bucket_name):
    """ Upload data to a bucket"""

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'bionic-genre-326715-416d6d51b533.json')

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [4]:
def delete_blob(blob_name, bucket_name):
    """Deletes a blob from the bucket."""

    storage_client = storage.Client.from_service_account_json(
        'bionic-genre-326715-416d6d51b533.json')

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

In [5]:
# Imports the Google Cloud client library
def speech_to_text(file_path):
    
    from google.cloud import speech

    gcs_uri=upload_to_bucket("MeetingOWL",file_path,"nlp_project_13")
    # Instantiates a client
    client = speech.SpeechClient()

    # The name of the audio file to transcribe
    gcs_uri = "gs://nlp_project_13/MeetingOWL"

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        #encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        encoding= speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        sample_rate_hertz=16000,
        language_code="en-IN",enable_word_time_offsets=True,enable_automatic_punctuation=True,
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)
    word_stamps=[]
    transcript=[]
    
    for result in response.results:
        alternative = result.alternatives[0]
        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time
            word_stamps.append([word,start_time,end_time])
        transcript.append(alternative.transcript)
    delete_blob("MeetingOWL","nlp_project_13")
    return [transcript,word_stamps]
   # return response.results 
#speech_to_text("finally.mp3")

### Writing the transcript on to a text file

In [6]:
def test_transcript(transcript):
    file1 = open('myfile.txt', 'w')
    file1.writelines(transcript)
    file1.close()

### Tokenization, lemmatization and stemming

In [7]:
stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

### Question detection 

In [8]:
class isQuestion():
    def __init__(self):
        posts = nltk.corpus.nps_chat.xml_posts()
        features = self.__get_feature_set(posts)
        self.classifier = self.naiveBayes(features)
    def __get_feature_set(self, posts):
        feature = []
        for post in posts:
            post_text = post.text            
            features = {}
            words = nltk.word_tokenize(post_text)
            for word in words:
                features['contains({})'.format(word.lower())] = True
            feature.append((features, post.get('class')))
        return feature
    def naiveBayes(self, feature_set):
        ## Creates a Naive Bayes model using the words and the tags of words present in the nps_chat corpus
        training_size = int(len(feature_set) * 0.1)
        train_set, test_set = feature_set[training_size:], feature_set[:training_size]
        classifier = nltk.NaiveBayesClassifier.train(train_set)
        return classifier
    def predict_question(self, text):
        ## Given a list of sentences, it returns a list of sentences which are questions using naive bayes model
        words = nltk.word_tokenize(text.lower())        
        if '?' in text:
            return 1
        features = {}
        for word in words:
            features['contains({})'.format(word.lower())] = True            
        prediction_result = self.classifier.classify(features)
        if prediction_result == 'whQuestion' or prediction_result == 'ynQuestion':
            return 1
        return 0

In [9]:
#return the list of questions
def Question_detection(transcript):
    result = []
    obj = isQuestion()
    for i in transcript:
        if(obj.predict_question(i) == 1):
            result.append(i)
    return result
    #transcript is a list of setences ex. ["hello everyone","how is everyone","lets start todays class"]
    

### Finding the timestamps of silence

In [10]:
def silience(file_name):
    from pydub import AudioSegment,silence
    myaudio = intro = AudioSegment.from_wav(file_name)
    silence = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=-16)
    silence = [((start/1000),(stop/1000)) for start,stop in silence] #convert to sec
    return silence

### Question difficulty classification

In [11]:
def ask_question(question,all_data,k,threshold,tfidf_vectorizer,tfidf_matrix):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)[0]
    sim=[]
    for i in range(len(similarity)):
        sim.append((similarity[i],i))
    sim.sort(reverse=True)
    #use the top k results 
    interesting,difficult = 0,0
    for max_indices in range(k):
        max_index=sim[max_indices][1]
        max_sim = sim[max_indices][0]
        weight_diff = -1
        weight_interesting = -1
        if(all_data.iloc[max_index]['Is the question difficult?']=='Yes'): weight_diff=1
        if(all_data.iloc[max_index]['Is the question interesting?']=='Yes'): weight_interesting=1
        difficult+= weight_diff*max_sim
        interesting+= weight_interesting*max_sim
    
    result={}
    
    if(difficult>=threshold):
        result["difficulty"]="Yes"
    else:
        result["difficulty"]="No"
    
    if(interesting>=threshold):
        result["interesting"]="Yes"
    else:
        result["interesting"]="No"
        
    return result
    

In [12]:
def interesting_questions(Questions,df,tfidf_vectorizer,tfidf_matrix):
    diff,inter=0,0
    for question in Questions:
        response = ask_question(question,df,1,0,tfidf_vectorizer,tfidf_matrix)
        if(response["difficulty"]=="Yes"): diff+=1
        if(response["interesting"]=="Yes"): inter+=1
    return [diff,inter]

In [13]:
def init_data():
    #path to the form responses
    df = pd.read_csv("question_feedback.csv")
    df.iloc[0:,1:]
    df=df.rename(columns={'Write a question relevant to our subjects which profs ask(eg. What is NLP?, Can someone tell me the difference between ML and AI? etc.)':'Question'})
    tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
    tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(df['Question']))
    return [df,tfidf_matrix,tfidf_vectorizer]

In [14]:
import anvil.media
from shutil import copy
@anvil.server.callable
def write_media_to_file(media_object):
    with anvil.media.TempFile(media_object) as f:
        #audio file stored in finally.mp3
        df,tfidf_matrix,tfidf_vectorizer=init_data()
        copy(f, "audio.mp3")
        #convert the speech to text
        transcript,word_stamps=speech_to_text("audio.mp3")
        Questions = Question_detection(transcript)
        diff,inter=interesting_questions(Questions,df,tfidf_vectorizer,tfidf_matrix)
        test_transcript(transcript)
        return [len(transcript),len(Questions),diff,inter]

In [22]:
def noOfSpeakers():
    from google.cloud import speech_v1p1beta1 as speech
    #from google.cloud.speech import enums
    client = speech.SpeechClient()
    speech_file = "audio.mpeg"
    with open(speech_file, "rb") as audio_file:
        content = audio_file.read()
    print("processing...")
    gcs_uri = "gs://nlp_project_13/audio.mpeg"
    #audio = speech.RecognitionAudio(content=content)
    audio = speech.RecognitionAudio(uri=gcs_uri)
    diarization_config = speech.SpeakerDiarizationConfig(
      enable_speaker_diarization=True,
      min_speaker_count=2,
      max_speaker_count=10,
    )

    config = speech.RecognitionConfig(
        encoding = speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        sample_rate_hertz=8000,
        language_code="en-US",
        diarization_config=diarization_config,
    )
    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=500)
    for result in response.results:
        words_info = result.alternatives[0].words
    speakers = []
    noOfSpeakers = 0
    for word_info in words_info:
        speakers.append(int(word_info.speaker_tag))
        #noOfSpeakers = max(noOfSpeakers, int(word_info.speaker_tag))
        print(
            u"word: '{}', speaker_tag: {}".format(word_info.word, word_info.speaker_tag)
        )
    speakers = set(speakers)
    return len(speakers)


### Driver Code

In [23]:
def demo():
    ## Speech to text
    print(noOfSpeakers())
    """text = speech_to_text("audio.mpeg")
    print("Transcript of the audio file is:\n")
    print(text)"""
    questions = Question_detection(["Hello, can you tell me what is machine learning?","machine learning is a branch of Artificial Intelligence and computer science which focuses on the use of data and algorithms to imitate the way humans learn","Can you tell me what are the different frameworks which are available in JavaScript?","Could you tell me the founders of Google?","the founders of Google are Larry Page and Sergey Brin."])
    print("\nList of questions from the transcript are:\n")
    print(questions)
    df_and_tfIdf = init_data()
    Interesting_and_diff_Questions = interesting_questions(questions, df_and_tfIdf[0], df_and_tfIdf[2], df_and_tfIdf[1])
    print("\nThe number of interesting questions from the list of questions are: " + str(Interesting_and_diff_Questions[0]))
    print("\nThe number of difficult questions from the list of questions are: " + str(Interesting_and_diff_Questions[1])) 

In [26]:
demo()

processing...
word: 'hello', speaker_tag: 2
word: 'can', speaker_tag: 2
word: 'you', speaker_tag: 2
word: 'tell', speaker_tag: 2
word: 'me', speaker_tag: 2
word: 'what', speaker_tag: 2
word: 'this', speaker_tag: 2
word: 'machine', speaker_tag: 2
word: 'line', speaker_tag: 2
word: 'machine', speaker_tag: 2
word: 'learning', speaker_tag: 2
word: 'is', speaker_tag: 2
word: 'a', speaker_tag: 2
word: 'branch', speaker_tag: 2
word: 'of', speaker_tag: 2
word: 'Activision', speaker_tag: 2
word: 'in', speaker_tag: 2
word: 'Pleasant', speaker_tag: 2
word: 'and', speaker_tag: 2
word: 'computer', speaker_tag: 2
word: 'science', speaker_tag: 2
word: 'which', speaker_tag: 2
word: 'focuses', speaker_tag: 2
word: 'on', speaker_tag: 2
word: 'the', speaker_tag: 2
word: 'use', speaker_tag: 2
word: 'of', speaker_tag: 2
word: 'data', speaker_tag: 2
word: 'structures', speaker_tag: 2
word: 'and', speaker_tag: 2
word: 'algorithms', speaker_tag: 2
word: 'can', speaker_tag: 2
word: 'you', speaker_tag: 2
word: 